In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input,CuDNNLSTM,LSTM
from keras.models import Model
from keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DATA/Just 8k - Sheet1.csv')
df.head()

,id,text,label
0,0,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন।,negative
1,1,এটা কেন হচ্ছে? সংশ্লিষ্ট সকলের ডিপ্রেশনের ফলে?...,negative
2,2,আমাদের দেশের স্বাভাবিক অর্থনৈতিক গতিপ্রবাহকে ব...,negative
3,3,চুরি নয় লুটপাট।,negative
4,4,ইসলামী ব্যাংকের বর্তমান অবস্থা দেখে মনে হয় শাস...,neutral


In [ ]:
!pip install bnlp_toolkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 4.8 MB/s 
     |████████████████████████████████| 23.9 MB 46.4 MB/s 
     |████████████████████████████████| 965 kB 73.6 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
from bnlp.corpus import stopwords, punctuations
from nltk.tokenize import word_tokenize
def remove_stop_words(text):
  word_tokens = word_tokenize(text)
  words = [w for w in word_tokens if not w in stopwords]
  return " ".join(words)
def preprocessing(text):
  text_backup = text  
  text = re.sub('[%s]' % re.escape(punctuations), ' ', text)
  text = "".join(i for i in text if i in ["।"] or 2432 <= ord(i) <= 2559 or ord(i)== 32)
  text = " ".join(text.split())
  text = re.sub('\n', ' ', text)
  text = remove_stop_words(text)
  if len(text) == 0:
    text = re.sub('[%s]' % re.escape(punctuations), ' ', text_backup)
    return remove_stop_words(text)
  return text
df['text'] = df['text'].apply(lambda x : preprocessing(str(x)))

punkt not found. downloading...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
df.head()

,id,text,label
0,0,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন,negative
1,1,সংশ্লিষ্ট সকলের ডিপ্রেশনের সরকার মনোনিত পরিচাল...,negative
2,2,দেশের স্বাভাবিক অর্থনৈতিক গতিপ্রবাহকে বাধাগ্রস...,negative
3,3,চুরি লুটপাট,negative
4,4,ইসলামী ব্যাংকের বর্তমান অবস্থা শাসক জামাতের সা...,neutral


In [ ]:
sentences = []
sum = 0
for comment in df['text']:
  sents = word_tokenize(comment.strip())
  sum += len(sents)
  for sent in sents:
    cleaned_sent = preprocessing(sent)
    sentences.append(cleaned_sent.split())
print(sum)
print(len(sentences))

107272
107272


In [ ]:
import gensim
w2v_model=gensim.models.Word2Vec(sentences=sentences,vector_size=300,window=5,min_count=1)

In [ ]:
vocab = w2v_model.wv.key_to_index
vocab=list(vocab.keys())

In [ ]:
word_vec_dict={}
for word in vocab:
  word_vec_dict[word]=w2v_model.wv.get_vector(word)

In [ ]:
from keras.preprocessing.text import one_hot,Tokenizer
tok = Tokenizer()
tok.fit_on_texts(df['text'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(df['text'])

In [ ]:
df.head()

,id,text,label
0,0,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন,negative
1,1,সংশ্লিষ্ট সকলের ডিপ্রেশনের সরকার মনোনিত পরিচাল...,negative
2,2,দেশের স্বাভাবিক অর্থনৈতিক গতিপ্রবাহকে বাধাগ্রস...,negative
3,3,চুরি লুটপাট,negative
4,4,ইসলামী ব্যাংকের বর্তমান অবস্থা শাসক জামাতের সা...,neutral


In [ ]:
df['label']=df['label'].replace({'positive':1,'negative':0,'neutral':2})
df.head()

,id,text,label
0,0,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন,0
1,1,সংশ্লিষ্ট সকলের ডিপ্রেশনের সরকার মনোনিত পরিচাল...,0
2,2,দেশের স্বাভাবিক অর্থনৈতিক গতিপ্রবাহকে বাধাগ্রস...,0
3,3,চুরি লুটপাট,0
4,4,ইসলামী ব্যাংকের বর্তমান অবস্থা শাসক জামাতের সা...,2


In [ ]:
maxi = -1
for i,rev in enumerate(df['text']):
  tokens=rev.split()
  if (len(tokens) > maxi):
    maxi = len(tokens)

In [ ]:
tok = Tokenizer()
tok.fit_on_texts(df['text'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(df['text'])

In [ ]:
max_rev_len = maxi + 1
vocab_size = len(tok.word_index) + 1
embed_dim=300

In [ ]:
pad_rev= pad_sequences(encd_rev, maxlen=max_rev_len, padding='post')
pad_rev.shape

(8066, 77)

In [ ]:
embed_matrix=np.zeros(shape=(vocab_size,embed_dim))
for word,i in tok.word_index.items():
  embed_vector=word_vec_dict.get(word)
  if embed_vector is not None:
    embed_matrix[i]=embed_vector

In [ ]:
Y = to_categorical(df['label'])
x_train,x_test,y_train,y_test=train_test_split(pad_rev,Y,test_size=0.20,random_state=42)

In [ ]:
classes = len(df['label'].value_counts()) #number of labels

In [ ]:
from tensorflow.keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.initializers import Constant
from keras.layers import ReLU
from tensorflow.keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.initializers import Constant
from keras.layers import ReLU
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras import models
from keras import layers
from tensorflow.keras.layers import LSTM

In [ ]:
embedding_vector_features=40
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=embed_dim,input_length=max_rev_len,embeddings_initializer=Constant(embed_matrix)),
    tf.keras.layers.SimpleRNN(32),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(3, activation='sigmoid')
])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 77, 300)           4856700   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                10656     
                                                                 
 dense_2 (Dense)             (None, 10)                330       
                                                                 
 dense_3 (Dense)             (None, 3)                 33        
                                                                 
Total params: 4,867,719
Trainable params: 4,867,719
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
101/101 [==============================] - 12s 94ms/step - loss: 0.6373 - accuracy: 0.3906 - val_loss: 0.6286 - val_accuracy: 0.4120
Epoch 2/10
101/101 [==============================] - 9s 91ms/step - loss: 0.6278 - accuracy: 0.3907 - val_loss: 0.6267 - val_accuracy: 0.4120
Epoch 3/10
101/101 [==============================] - 9s 90ms/step - loss: 0.6285 - accuracy: 0.3937 - val_loss: 0.6394 - val_accuracy: 0.3618
Epoch 4/10
101/101 [==============================] - 9s 91ms/step - loss: 0.6274 - accuracy: 0.4076 - val_loss: 0.6181 - val_accuracy: 0.4504
Epoch 5/10
101/101 [==============================] - 9s 90ms/step - loss: 0.6109 - accuracy: 0.4744 - val_loss: 0.5812 - val_accuracy: 0.5347
Epoch 6/10
101/101 [==============================] - 9s 90ms/step - loss: 0.5305 - accuracy: 0.6135 - val_loss: 0.5765 - val_accuracy: 0.5582
Epoch 7/10
101/101 [==============================] - 9s 90ms/step - loss: 0.4095 - accuracy: 0.7212 - val_loss: 0.5231 - val_accuracy: 0.633

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("RNN accuracy = ",scores[1]*100)

RNN accuracy =  74.41139817237854


In [ ]:
prediction= (model.predict(x_test))
prediction

array([[0.9393517 , 0.00329861, 0.05435678],
       [0.22534952, 0.00568515, 0.7335785 ],
       [0.00454512, 0.1675351 , 0.8904704 ],
       ...,
       [0.01553941, 0.39157596, 0.67196226],
       [0.01481658, 0.4099233 , 0.65420675],
       [0.00420052, 0.9301218 , 0.05417344]], dtype=float32)

In [ ]:
test=np.argmax(y_test,axis=1)
pred=np.argmax(prediction,axis=1)

In [ ]:
from sklearn import datasets
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
 

 
from sklearn.metrics import classification_report

print(classification_report(test,pred))

              precision    recall  f1-score   support

           0       0.82      0.82      0.82       665
           1       0.76      0.76      0.76       547
           2       0.60      0.60      0.60       402

    accuracy                           0.74      1614
   macro avg       0.73      0.73      0.73      1614
weighted avg       0.74      0.74      0.74      1614



In [ ]:
import sklearn
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
print(metrics.confusion_matrix(test,pred))

[[545  45  75]
 [ 47 413  87]
 [ 72  87 243]]


In [ ]:
class_names=['Negative','Positive', 'Neutral']

In [ ]:
import seaborn as sns
def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True Label')
  plt.xlabel('Predicted Label');
cm = confusion_matrix(test, pred)
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
show_confusion_matrix(df_cm)